## **DATA PRE-PROCESSING**

### **Load Data**

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving FULL_DATA.csv to FULL_DATA.csv


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['FULL_DATA.csv']),parse_dates=["DATE"])
data.head()


In [ ]:
data.info()

In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from scipy.stats.mstats import zscore
import seaborn as sns
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.stattools import adfuller
import itertools
import math
import random


In [ ]:
df=data.copy()

### **Missing Values**

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
mis_val= missing_values_table(df)
mis_val

In [ ]:
# Delete columns containing either 10% or more than 10% NaN Values
perc = 10.0
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
mod_df = df.dropna( axis=1, 
                thresh=min_count)

In [ ]:
mod_df.shape

(213, 43)

In [ ]:
# Add a Quarterly variables back to DataFrame
q = df['ACTIVEJOBS']
q1 = df['JOBVACAN']
q2 = df['RETRENCH']
q3 = df['GDPGR']
mod_df = mod_df.join(q)
mod_df = mod_df.join(q1)
mod_df = mod_df.join(q2)
mod_df = mod_df.join(q3)

In [ ]:
mod_df

### **Missing Values - MICE**

In [ ]:
#MICE
import numpy as np 
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_2 = mod_df.copy()
df_3 = df_2.drop(['DATE'],axis=1)

imputer=IterativeImputer(imputation_order='ascending',max_iter=10,random_state=42,n_nearest_features=5)
imputer
imputed_dataset = imputer.fit_transform(df_3)

df_3 = pd.DataFrame(imputed_dataset, columns = ['ALRCB','CIC','CPI','CRUDEBRUSD','EXSITC0','EXSITC1','EXSITC2','EXSITC3','EXSITC4','EXSITC5','EXSITC6','EXSITC7','EXSITC8','EXSITC9','EXTOT','IMSITC0','IMSITC1','IMSITC2','IMSITC3','IMSITC4','IMSITC5','IMSITC6','IMSITC7','IMSITC8','IMSITC9','IMTOT','IPIELEC','IPIMFG','IPIMIN','IPITOT','LGC','LIC','TOURIST','USDEXC','SOPC','SALETAX','SERVTAX','CONSCRE','RUBBER','PALMOIL','IMPCON','OBOCC','ACTIVEJOBS','JOBVACAN','RETRENCH','GDPGR'])

print(df_3)
print(type(df_3))

df_3.head()

In [ ]:
df_3.isnull().sum()

In [ ]:
#Adding back DATE column

date = df['DATE']
df_3 = df_3.join(date)

df_3.head()

In [ ]:
df_3.head()

In [ ]:
from google.colab import files
df_3.to_csv('GDPGR_clean.csv') 
files.download('GDPGR_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Transformation - Federal reserve NY**

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving GDPGR_clean.csv to GDPGR_clean.csv


In [ ]:
%matplotlib inline
import io
import types
import numpy as np
import pandas as pd
#import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
def transform(column, transforms):
    transformation = transforms[column.name]
    # For quarterly data like GDP, we will compute
    # annualized percent changes
    mult = 4 if column.index.freqstr[0] == 'Q' else 1
    
    # 1 => No transformation
    if transformation == 1:
        pass
    # 2 => First difference
    elif transformation == 2:
        column = column.diff()
    # 3 => Second difference
    elif transformation == 3:
        column = column.diff().diff()
    # 4 => Log
    elif transformation == 4:
        column = np.log(column)
    # 5 => Log first difference, multiplied by 100
    #      (i.e. approximate percent change)
    elif transformation == 5:
        column = np.log(column).diff() * 100
    # 6 => Log second difference, multiplied by 100
    elif transformation == 6:
        column = np.log(column).diff().diff() * 100
    # 7 => Exact percent change, multiplied by 100
    #      with optional annualization
    elif transformation == 7:
        column = ((column / column.shift(1))**mult - 1.0) * 100
    # 8 => Log first difference, multiplied by 100
    #      (i.e. approximate percent change)
    #      with optional multiplier for annualization
    elif transformation == 8:
        column = np.log(column).diff() * 100 * mult
    # 9 => Log second difference, multiplied by 100
    #      with optional multiplier for annualization
    elif transformation == 9:
        column = np.log(column).diff().diff() * 100 * mult
    # 10 => Exact percent change, multiplied by 100
    #      with optional annualization
    elif transformation == 10:
        column = ((column / column.shift(1)) - 1.0) * 100
    elif transformation == 11:
        column = np.arcsinh(column).diff() * 100
    elif transformation == 12:
        column = np.arccosh(column).diff() * 100
    elif transformation == 13:
        column = np.arctanh(column).diff() * 100

      
    return column

In [ ]:
def load_transformation():
        
    
    # 1. Download data
    #orig_m = (pd.read_csv(f'{base_url}/monthly/{vintage}.csv').dropna(how='all'))
    orig_m = (pd.read_csv(io.BytesIO(data_to_load['GDPGR_clean.csv']),parse_dates=["DATE"]))
    
    # 2. Extract transformation information
    transform_m = orig_m.iloc[0, 1:]
    orig_m = orig_m.iloc[1:]

    # 3. Extract the date as an index
    orig_m.index = pd.PeriodIndex(orig_m.DATE.tolist(), freq='M')
    orig_m.drop('DATE', axis=1, inplace=True)

    # 4. Apply the transformations
    dta_m = orig_m.apply(transform, axis=0, transforms=transform_m)

    orig_m=orig_m
    dta_m=dta_m
    transform_m=transform_m

    df =pd.DataFrame(dta_m)

    #return types.SimpleNamespace(orig_m=orig_m, dta_m=dta_m, )    

    return df

df = load_transformation()
df
      

In [ ]:
df.isnull().sum()

In [ ]:
df_tra = df.copy()
df_tra.shape

In [ ]:
# making new data frame with dropped NA values 
new_df = df_tra.dropna(axis = 0, how ='any') 
    
new_df

In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['gdp_yoy.csv']),parse_dates=["DATE"])
data.head()

In [ ]:
df= data.copy()

In [ ]:
# creating features and label variable
X = df[['ALRCB','CIC','CPI','CRUDEBRUSD','EXSITC0','EXSITC1','EXSITC2','EXSITC3','EXSITC4','EXSITC5','EXSITC6','EXSITC7','EXSITC8','EXSITC9','EXTOT','IMSITC0','IMSITC1','IMSITC2','IMSITC3','IMSITC4','IMSITC5','IMSITC6','IMSITC7','IMSITC8','IMSITC9','IMTOT','IPIELEC','IPIMFG','IPIMIN','IPITOT','LGC','LIC','TOURIST','USDEXC','SOPC','SALETAX','SERVTAX','CONSCRE','RUBBER','PALMOIL','IMPCON','OBOCC','RETRENCH','ACTIVEJOBS','JOBVACAN']]
y = df[["GDPGR"]]

In [ ]:
# Function to check stationarity

def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [ ]:
# Data Prep

AI_Model_data = pd.concat([y, X], axis=1, sort=False)
#AI_Model_data = AI_Model_data.dropna()
# Check stationarity

# ADF Test on each column
for name, column in AI_Model_data.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

In [ ]:
#Save transformation
from google.colab import files
df.to_csv('gdp_arcsinh.csv') 
files.download('gdp_arcsinh.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['gdp_arcsinh.csv']),parse_dates=["DATE"])
data.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,2], name = data.iloc[:,2].name, line = dict(color = '#FF0000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,3], name = data.iloc[:,3].name, line = dict(color = '#CD853F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,4], name = data.iloc[:,4].name, line = dict(color = '#483D8B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,5], name = data.iloc[:,5].name, line = dict(color = '#00008B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,6], name = data.iloc[:,6].name, line = dict(color = '#8B4513'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,7], name = data.iloc[:,7].name, line = dict(color = '#7B68EE'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,8], name = data.iloc[:,8].name, line = dict(color = '#800080'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,9], name = data.iloc[:,9].name, line = dict(color = '#87CEEB'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,10], name = data.iloc[:,10].name, line = dict(color = '#FFDAB9'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,11], name = data.iloc[:,11].name, line = dict(color = '#32CD32'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,12], name = data.iloc[:,12].name, line = dict(color = '#D8BFD8'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,13], name = data.iloc[:,13].name, line = dict(color = '#DDA0DD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,14], name = data.iloc[:,14].name, line = dict(color = '#6495ED'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,15], name = data.iloc[:,15].name, line = dict(color = '#FFA07A'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,16], name = data.iloc[:,16].name, line = dict(color = '#4682B4'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,17], name = data.iloc[:,17].name, line = dict(color = '#556B2F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,18], name = data.iloc[:,18].name, line = dict(color = '#9ACD32'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,19], name = data.iloc[:,19].name, line = dict(color = '#B0C4DE'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,20], name = data.iloc[:,20].name, line = dict(color = '#9370DB'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,21], name = data.iloc[:,21].name, line = dict(color = '#DAA520'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,22], name = data.iloc[:,22].name, line = dict(color = '#0000CD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,23], name = data.iloc[:,23].name, line = dict(color = '#008080'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,24], name = data.iloc[:,24].name, line = dict(color = '#B8860B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,25], name = data.iloc[:,25].name, line = dict(color = '#FFD700'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,26], name = data.iloc[:,26].name, line = dict(color = '#800000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,27], name = data.iloc[:,27].name, line = dict(color = '#FF8C00'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,28], name = data.iloc[:,28].name, line = dict(color = '#B0E0E6'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,29], name = data.iloc[:,29].name, line = dict(color = '#FF00FF'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,30], name = data.iloc[:,30].name, line = dict(color = '#F4A460'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,31], name = data.iloc[:,31].name, line = dict(color = '#FFFF00'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,32], name = data.iloc[:,32].name, line = dict(color = '#D2691E'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,33], name = data.iloc[:,33].name, line = dict(color = '#7FFFD4'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,34], name = data.iloc[:,34].name, line = dict(color = '#0000FF'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,35], name = data.iloc[:,35].name, line = dict(color = '#FFEFD5'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,36], name = data.iloc[:,36].name, line = dict(color = '#EEE8AA'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,37], name = data.iloc[:,37].name, line = dict(color = '#FFA500'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,38], name = data.iloc[:,38].name, line = dict(color = '#9400D3'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,39], name = data.iloc[:,39].name, line = dict(color = '#FFDEAD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,40], name = data.iloc[:,40].name, line = dict(color = '#66CDAA'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,41], name = data.iloc[:,41].name, line = dict(color = '#8B008B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,42], name = data.iloc[:,42].name, line = dict(color = '#D2B48C'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,43], name = data.iloc[:,43].name, line = dict(color = '#ADFF2F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,44], name = data.iloc[:,44].name, line = dict(color = '#008000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,45], name = data.iloc[:,45].name, line = dict(color = '#F5DEB3'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,46], name = data.iloc[:,46].name, line = dict(color = '#000000'), opacity = 1.0))